## GGPlot Tutorial


In [ ]:
import hail as hl
from hail.ggplot import *

import plotly
import plotly.io as pio
pio.renderers.default='iframe'

The Hail team has implemented a plotting module for hail based on the very popular `ggplot2` package from R's tidyverse. That library is very fully featured and we will never be quite as flexible as it, but with just a subset of its functionality we can make highly customizable plots.

### The Grammar of Graphics

The key idea here is that there's not one magic function to make the plot you want. Plots are built up from a set of core primitives that allow for extensive customization. Let's start with an example. We are going to plot y = x^2 for x from 0 to 10. First we make a hail table representing that data:

In [ ]:
ht = hl.utils.range_table(10)
ht = ht.annotate(squared = ht.idx**2)

Every plot starts with a call to `ggplot`, and then requires adding a `geom` to specify what kind of plot you'd like to create.

In [ ]:
fig = ggplot(ht, aes(x=ht.idx, y=ht.squared)) + geom_line()
fig.show()

`aes` creates an "aesthetic mapping", which maps hail expressions to aspects of the plot. There is a predefined list of aesthetics supported by every `geom`. Most take an `x` and `y` at least. 

With this interface, it's easy to change out our plotting representation separate from our data. We can plot bars:

In [ ]:
fig = ggplot(ht, aes(x=ht.idx, y=ht.squared)) + geom_col()
fig.show()

Or points:

In [ ]:
fig = ggplot(ht, aes(x=ht.idx, y=ht.squared)) + geom_point()
fig.show()

There are optional aesthetics too. If we want, we could color the points based on whether they're even or odd:

In [ ]:
fig = ggplot(ht, aes(x=ht.idx, y=ht.squared, color=hl.if_else(ht.idx % 2 == 0, "even", "odd"))) + geom_point()
fig.show()

Note that the `color` aesthetic by default just takes in an expression that evaluates to strings, and it assigns a discrete color to each string.

Say we wanted to plot the line with the colored points overlayed on top of it. We could try:

In [ ]:
fig = (ggplot(ht, aes(x=ht.idx, y=ht.squared, color=hl.if_else(ht.idx % 2 == 0, "even", "odd"))) + 
       geom_line() + 
       geom_point() 
      )
fig.show()

But that is coloring the line as well, causing us to end up with interlocking blue and orange lines, which isn't what we want. For that reason, it's possible to define aesthetics that only apply to certain geoms.

In [ ]:
fig = (ggplot(ht, aes(x=ht.idx, y=ht.squared)) + 
       geom_line() + 
       geom_point(aes(color=hl.if_else(ht.idx % 2 == 0, "even", "odd"))) 
      )
fig.show()

All geoms can take in their own aesthetic mapping, which lets them specify aesthetics specific to them. And `geom_point` still inherits the `x` and `y` aesthetics from the mapping defined in `ggplot()`.

### Geoms that group

Some geoms implicitly do an aggregation based on the `x` aesthetic, and so don't take a `y` value. Consider this dataset from gapminder with information about countries around the world, with one datapoint taken per country in the years 1952, 1957, 1962, 1967, 1972, 1977, 1982, 1987, 1992, 1997, 2002, and 2007.

In [ ]:
gp = hl.Table.from_pandas(plotly.data.gapminder())
gp.describe()

Let's filter the data to 2007 for our first experiments

In [ ]:
gp_2007 = gp.filter(gp.year == 2007)

If we want to see how many countries from each continent we have, we can use `geom_bar`, which just takes in an x aesthetic and then implicitly counts how many values of each x there are.

In [ ]:
ggplot(gp_2007, aes(x=gp_2007.continent)) + geom_bar()

To make it a little prettier, let's color per continent as well. We use `fill` to specify color of shapes (as opposed to `color` for points and lines. `color` on a bar chart sets the color of the bar outline.)

In [ ]:
ggplot(gp_2007, aes(x=gp_2007.continent)) + geom_bar(aes(fill=gp_2007.continent))

Maybe we instead want to see not the number of countries per continent, but the number of people living on each continent. We can do this with `geom_bar` as well by specifying a weight.

In [ ]:
ggplot(gp_2007, aes(x=gp_2007.continent)) + geom_bar(aes(fill=gp_2007.continent, weight=gp_2007.pop))

Histograms are similar to bar plots, except they break a continuous x axis into bins. Let's import the `iris` dataset for this.


In [ ]:
iris = hl.Table.from_pandas(plotly.data.iris())
iris.describe()

Let's make a histogram:

In [ ]:
ggplot(iris, aes(x=iris.sepal_length, fill=iris.species)) + geom_histogram()

By default histogram plots groups stacked on top of each other, which is not always easy to interpret. We can specify the `position` argument to histogram to get different behavior. `"dodge"` puts the bars next to each other:

In [ ]:
ggplot(iris, aes(x=iris.sepal_length, fill=iris.species)) + geom_histogram(position="dodge")

And `"identity"` plots them over each other. It helps to set an `alpha` value to make them slightly transparent in these cases

In [ ]:
ggplot(iris, aes(x=iris.sepal_length, fill=iris.species)) + geom_histogram(position="identity", alpha=0.8)

### Labels and Axes

It's always a good idea to label your axes. This can be done most easily with `xlab` and `ylab`. We can also use `ggtitle` to add a title. Let's pull in the same plot from above, and add labels.

In [ ]:
(ggplot(iris, aes(x=iris.sepal_length, fill=iris.species)) + 
 geom_histogram(position="identity", alpha=0.8) + 
 xlab("Sepal Length") + ylab("Number of samples") + ggtitle("Sepal length by flower type")
)